Cargar Data y primeros analisis

In [1]:
import bootcampviztools as bt
import hashlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import modulo_toolbox as mtb
import requests
import seaborn as sns
import time
import toolbox_ML as tb
import warnings

warnings.filterwarnings('ignore')

In [2]:
df_DC = pd.read_csv('./data/heroes_dc_extended')

In [3]:
df_DC.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Unnamed: 0       50 non-null     int64 
 1   nombre_completo  39 non-null     object
 2   nombre           50 non-null     object
 3   universo         50 non-null     object
 4   url              50 non-null     object
 5   Intelligence     50 non-null     int64 
 6   Strength         50 non-null     int64 
 7   Speed            50 non-null     int64 
 8   Durability       50 non-null     int64 
 9   Power            50 non-null     int64 
 10  Combat           50 non-null     int64 
 11  Tier             50 non-null     int64 
 12  Tier2            6 non-null      object
 13  heroe            50 non-null     object
dtypes: int64(8), object(6)
memory usage: 5.6+ KB


In [4]:
# Mostrar las primeras filas del dataframe para inspeccionar
df_DC.head(20)

,Unnamed: 0,nombre_completo,nombre,universo,url,Intelligence,Strength,Speed,Durability,Power,Combat,Tier,Tier2,heroe
0,0,Alfred Pennyworth,Alfred,DC Extended Universe,https://www.superherodb.com/alfred/10-14275/#shdb,65,1,1,5,1,20,0,NaN,Alfred
1,1,Amanda Blake Waller,Amanda Waller,DC Extended Universe,https://www.superherodb.com/amanda-waller/10-3...,65,1,1,5,10,30,0,NaN,Amanda Waller
2,2,NaN,Anthea,DC Extended Universe,https://www.superherodb.com/anthea/10-54768/#user,0,0,0,0,0,0,0,NaN,Anthea
3,3,Arthur Curry,Aquaman,DC Extended Universe,https://www.superherodb.com/aquaman/10-12533/,65,50,25,30,45,80,3,NaN,Aquaman
4,4,Ares,Ares,DC Extended Universe,https://www.superherodb.com/ares/10-12597/,90,100,50,50,70,100,4,4,Ares
5,5,Albert Rothstein,Atom Smasher,DC Extended Universe,https://www.superherodb.com/atom-smasher/10-27...,60,30,25,25,40,20,2,NaN,Atom Smasher
6,6,Bruce Wayne,Batman,DC Extended Universe,https://www.superherodb.com/batman/10-12530/,80,13,11,15,20,95,1,NaN,Batman
7,7,Teth-Adam,Black Adam,DC Extended Universe,https://www.superherodb.com/black-adam/10-21003/,65,100,75,45,75,75,3,3,Black Adam
8,8,Dinah Lance,Black Canary,DC Extended Universe,https://www.superherodb.com/black-canary/10-18...,55,1,6,5,35,40,1,NaN,Black Canary
9,9,David Kane,Black Manta,DC Extended Universe,https://www.superherodb.com/black-manta/10-15579/,70,25,25,20,40,60,2,NaN,Black Manta


Fuerza total de cada héroe basada en sus atributos (suma)

In [5]:
# Reemplazar los valores nulos con 0 para asegurar que la suma no falle
attributes = ['Intelligence', 'Strength', 'Speed', 'Durability', 'Power', 'Combat']
df_DC[attributes] = df_DC[attributes].fillna(0)

In [6]:
# Calcular la fuerza total
df_DC['Fuerza_Total'] = df_DC[attributes].sum(axis=1)

In [7]:
# Mostrar las primeras filas para verificar el cálculo
df_DC[['nombre', 'Intelligence', 'Strength', 'Speed', 'Durability', 'Power', 'Combat', 'Fuerza_Total']].head()

,nombre,Intelligence,Strength,Speed,Durability,Power,Combat,Fuerza_Total
0,Alfred,65,1,1,5,1,20,93
1,Amanda Waller,65,1,1,5,10,30,112
2,Anthea,0,0,0,0,0,0,0
3,Aquaman,65,50,25,30,45,80,295
4,Ares,90,100,50,50,70,100,460


Nuevo data con batallas

In [8]:
from itertools import combinations

# Crear combinaciones de pares de héroes
pairs = list(combinations(df_DC.index, 2))

# Crear un nuevo DataFrame para los pares de héroes y resultados de batallas
battle_data = []

for i, j in pairs:
    hero1 = df_DC.loc[i]
    hero2 = df_DC.loc[j]
    fuerza1 = hero1['Fuerza_Total']
    fuerza2 = hero2['Fuerza_Total']
    
    # Determinar el resultado de la batalla
    if fuerza1 > fuerza2:
        resultado = 'Gana'
    elif fuerza1 < fuerza2:
        resultado = 'Pierde'
    else:
        resultado = 'Empata'
    
    battle_data.append({
        'Heroe_1': hero1['nombre'],
        'Fuerza_Heroe_1': fuerza1,
        'Heroe_2': hero2['nombre'],
        'Fuerza_Heroe_2': fuerza2,
        'Resultado': resultado
    })

In [9]:
# Convertir la lista de batallas en un DataFrame
battle_df = pd.DataFrame(battle_data)

# Mostrar las primeras filas del DataFrame de batallas
battle_df.head(50)

,Heroe_1,Fuerza_Heroe_1,Heroe_2,Fuerza_Heroe_2,Resultado
0,Alfred,93,Amanda Waller,112,Pierde
1,Alfred,93,Anthea,0,Gana
2,Alfred,93,Aquaman,295,Pierde
3,Alfred,93,Ares,460,Pierde
4,Alfred,93,Atom Smasher,200,Pierde
5,Alfred,93,Batman,234,Pierde
6,Alfred,93,Black Adam,435,Pierde
7,Alfred,93,Black Canary,142,Pierde
8,Alfred,93,Black Manta,240,Pierde
9,Alfred,93,Black Mask,107,Pierde


Mapeo atributos heroes en dataframe batallas

In [10]:
# Vamos a añadir las columnas de atributos para cada héroe
hero_attributes = ['Intelligence', 'Strength', 'Speed', 'Durability', 'Power', 'Combat']

In [11]:
# Unir los atributos de Héroe_1
for attr in hero_attributes:
    battle_df[f'Heroe_1_{attr}'] = battle_df['Heroe_1'].map(df_DC.set_index('nombre')[attr])

# Unir los atributos de Héroe_2
for attr in hero_attributes:
    battle_df[f'Heroe_2_{attr}'] = battle_df['Heroe_2'].map(df_DC.set_index('nombre')[attr])

In [12]:
# Convertir el resultado a valores numéricos
result_mapping = {'Gana': 1, 'Pierde': 0, 'Empata': 2}
battle_df['Resultado_Numerico'] = battle_df['Resultado'].map(result_mapping)

In [13]:
# Mostrar las primeras filas del DataFrame procesado
battle_df.head()

,Heroe_1,Fuerza_Heroe_1,Heroe_2,Fuerza_Heroe_2,Resultado,Heroe_1_Intelligence,Heroe_1_Strength,Heroe_1_Speed,Heroe_1_Durability,Heroe_1_Power,Heroe_1_Combat,Heroe_2_Intelligence,Heroe_2_Strength,Heroe_2_Speed,Heroe_2_Durability,Heroe_2_Power,Heroe_2_Combat,Resultado_Numerico
0,Alfred,93,Amanda Waller,112,Pierde,65,1,1,5,1,20,65,1,1,5,10,30,0
1,Alfred,93,Anthea,0,Gana,65,1,1,5,1,20,0,0,0,0,0,0,1
2,Alfred,93,Aquaman,295,Pierde,65,1,1,5,1,20,65,50,25,30,45,80,0
3,Alfred,93,Ares,460,Pierde,65,1,1,5,1,20,90,100,50,50,70,100,0
4,Alfred,93,Atom Smasher,200,Pierde,65,1,1,5,1,20,60,30,25,25,40,20,0


In [14]:
df_DC[df_DC['nombre'] == 'Alfred']

,Unnamed: 0,nombre_completo,nombre,universo,url,Intelligence,Strength,Speed,Durability,Power,Combat,Tier,Tier2,heroe,Fuerza_Total
0,0,Alfred Pennyworth,Alfred,DC Extended Universe,https://www.superherodb.com/alfred/10-14275/#shdb,65,1,1,5,1,20,0,NaN,Alfred,93


In [15]:
df_DC[df_DC['nombre'] == 'Amanda Waller']

,Unnamed: 0,nombre_completo,nombre,universo,url,Intelligence,Strength,Speed,Durability,Power,Combat,Tier,Tier2,heroe,Fuerza_Total
1,1,Amanda Blake Waller,Amanda Waller,DC Extended Universe,https://www.superherodb.com/amanda-waller/10-3...,65,1,1,5,10,30,0,NaN,Amanda Waller,112


In [16]:
battle_df.iloc[150:200]

,Heroe_1,Fuerza_Heroe_1,Heroe_2,Fuerza_Heroe_2,Resultado,Heroe_1_Intelligence,Heroe_1_Strength,Heroe_1_Speed,Heroe_1_Durability,Heroe_1_Power,Heroe_1_Combat,Heroe_2_Intelligence,Heroe_2_Strength,Heroe_2_Speed,Heroe_2_Durability,Heroe_2_Power,Heroe_2_Combat,Resultado_Numerico
150,Aquaman,295,Black Mask,107,Gana,65,50,25,30,45,80,60,1,6,5,5,30,1
151,Aquaman,295,Bloodsport,199,Gana,65,50,25,30,45,80,70,1,8,15,20,85,1
152,Aquaman,295,Captain Boomerang,109,Gana,65,50,25,30,45,80,45,1,6,7,10,40,1
153,Aquaman,295,Cassandra Cain,79,Gana,65,50,25,30,45,80,45,1,1,1,1,30,1
154,Aquaman,295,Cheetah,234,Gana,65,50,25,30,45,80,60,50,14,15,30,65,1
155,Aquaman,295,Cyborg,290,Gana,65,50,25,30,45,80,70,40,30,35,65,50,1
156,Aquaman,295,Cyclone,211,Gana,65,50,25,30,45,80,80,1,25,15,50,40,1
157,Aquaman,295,Dark Flash,375,Pierde,65,50,25,30,45,80,80,25,100,35,85,50,0
158,Aquaman,295,Darkseid,465,Pierde,65,50,25,30,45,80,95,100,30,55,90,95,0
159,Aquaman,295,Darla Dudley,0,Gana,65,50,25,30,45,80,0,0,0,0,0,0,1


In [17]:
battle_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1225 entries, 0 to 1224
Data columns (total 18 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Heroe_1               1225 non-null   object
 1   Fuerza_Heroe_1        1225 non-null   int64 
 2   Heroe_2               1225 non-null   object
 3   Fuerza_Heroe_2        1225 non-null   int64 
 4   Resultado             1225 non-null   object
 5   Heroe_1_Intelligence  1225 non-null   int64 
 6   Heroe_1_Strength      1225 non-null   int64 
 7   Heroe_1_Speed         1225 non-null   int64 
 8   Heroe_1_Durability    1225 non-null   int64 
 9   Heroe_1_Power         1225 non-null   int64 
 10  Heroe_1_Combat        1225 non-null   int64 
 11  Heroe_2_Intelligence  1225 non-null   int64 
 12  Heroe_2_Strength      1225 non-null   int64 
 13  Heroe_2_Speed         1225 non-null   int64 
 14  Heroe_2_Durability    1225 non-null   int64 
 15  Heroe_2_Power         1225 non-null   

In [18]:
battle_df.to_csv('batallas_Dc')